# Tutorial 6: Denotational Semantics

*Giving Meaning to Lambda Terms*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/06-denotational-semantics.ipynb)

---

## A Note from the Archives

> *Year 880, Capital Archives*
>
> Arren Mott had spent twenty years building the machinery. Now came the payoff.
>
> "Let ⟦M⟧ρ denote the meaning of expression M in environment ρ," he wrote.
>
> The rules were elegant:
>
> - ⟦x⟧ρ = ρ(x)
> - ⟦λx.M⟧ρ = λd.⟦M⟧ρ[x↦d]
> - ⟦M N⟧ρ = ⟦M⟧ρ(⟦N⟧ρ)
>
> "Every expression has a meaning. Every meaning is an element of a domain. And the meaning of a recursive definition is its least fixed point."
>
> This was EV-880-001: the discovery of denotational semantics.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define denotational semantics and its purpose
2. Apply the semantic function ⟦M⟧ρ to lambda terms
3. Interpret environments and function meanings
4. State and understand the adequacy theorem
5. Explain how denotational and operational semantics relate

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

denotations_df = pd.read_csv(BASE_URL + "denotational_semantics.csv")
functions_df = pd.read_csv(BASE_URL + "continuous_functions.csv")

print(f"Loaded {len(denotations_df)} denotations")

## 1. What is Denotational Semantics?

**Denotational semantics** gives mathematical meaning to programs.

Instead of describing computation step-by-step (operational semantics), we describe the **final result** — what value the program denotes.

| Approach | Question | Answer |
|----------|----------|--------|
| Operational | How does M compute? | M → M' → M'' → ... → v |
| Denotational | What does M mean? | ⟦M⟧ρ = d ∈ D |

In [ ]:
# Compare operational and denotational views
print("Two Views of Computation:")
print()
print("  Expression: (λx.x) 5")
print()
print("  Operational: (λx.x) 5 → 5")
print("    (shows the reduction step)")
print()
print("  Denotational: ⟦(λx.x) 5⟧ρ = 5")
print("    (directly states the meaning)")

## 2. The Semantic Function ⟦M⟧ρ

The **semantic function** maps expressions to domain elements:

$$⟦ \cdot ⟧ : \text{Term} \to \text{Env} \to D$$

Where:
- Term = lambda expressions
- Env = environments (mappings from variables to values)
- D = a suitable domain

The function is defined inductively on the structure of terms.

In [ ]:
# The three rules of denotational semantics
print("The Semantic Equations:")
print()
print("  1. Variables:   ⟦x⟧ρ = ρ(x)")
print("     (look up x in the environment)")
print()
print("  2. Abstraction: ⟦λx.M⟧ρ = λd ∈ D. ⟦M⟧ρ[x↦d]")
print("     (a function that, given d, evaluates M with x=d)")
print()
print("  3. Application: ⟦M N⟧ρ = ⟦M⟧ρ (⟦N⟧ρ)")
print("     (apply the meaning of M to the meaning of N)")

## 3. Examples: Computing Denotations

Let's compute denotations for several expressions.

In [ ]:
# Show denotations from the dataset
basic_terms = denotations_df[denotations_df['expression'].str.len() < 30]

print("Denotations of Lambda Terms:")
print()
print("| Expression | Denotation | Terminates |")
print("|------------|------------|------------|")

for _, row in basic_terms.head(10).iterrows():
    term = row['terminates']
    status = "Yes" if term else "No"
    print(f"| {row['expression'][:25]:<25} | {row['denotation'][:25]:<25} | {status} |")

In [ ]:
# Trace the denotation of the identity function
print("Example: Denotation of Identity")
print()
print("  ⟦λx.x⟧ρ")
print("  = λd ∈ D. ⟦x⟧ρ[x↦d]      (by abstraction rule)")
print("  = λd ∈ D. ρ[x↦d](x)       (by variable rule)")
print("  = λd ∈ D. d               (x maps to d)")
print()
print("  The identity function denotes the identity function!")

## 4. The Meaning of Non-Termination

This is where domains earn their keep:

**Non-terminating expressions have ⊥ as their denotation.**

This gives mathematical meaning to programs that loop forever.

In [ ]:
# Non-terminating expressions
non_term = denotations_df[denotations_df['is_bottom'] == True]

print("Non-terminating expressions denote ⊥:")
print()
for _, row in non_term.iterrows():
    print(f"  ⟦{row['expression']}⟧ρ = ⊥")
    print(f"    Note: {row['notes']}")
    print()

In [ ]:
# Why Omega denotes bottom
print("Why does Omega denote ⊥?")
print()
print("  Let ω = λx.x x")
print("  Let Ω = ω ω")
print()
print("  ⟦Ω⟧ρ = ⟦ω ω⟧ρ")
print("       = ⟦ω⟧ρ (⟦ω⟧ρ)")
print()
print("  But ⟦ω⟧ρ = λd. d(d)")
print()
print("  So ⟦Ω⟧ρ = (λd. d(d))(λd. d(d))")
print("          = ⟦Ω⟧ρ    ← circular!")
print()
print("  The only consistent assignment is ⟦Ω⟧ρ = ⊥")

## 5. The Meaning of Recursion

For recursive definitions, we use the fixed point construction:

$$⟦\text{fix } f⟧ρ = \bigsqcup_{n \geq 0} ⟦f⟧ρ^n(\bot)$$

The recursive function's meaning IS the least fixed point.

In [ ]:
# Recursive definitions via fix
print("Denotational Semantics of Recursion:")
print()
print("  Recursive definition: factorial = λn. if n=0 then 1 else n*factorial(n-1)")
print()
print("  As fixed point: factorial = fix(F)")
print("    where F = λf.λn. if n=0 then 1 else n*f(n-1)")
print()
print("  Denotation: ⟦factorial⟧ρ = ⊔{Fⁿ(⊥) | n ≥ 0}")
print()
print("  The meaning of factorial IS the limit of its approximations.")

## 6. The Adequacy Theorem

The crucial connection between operational and denotational semantics:

**Theorem** (Adequacy): For closed terms M of ground type:

$$M \Downarrow v \iff ⟦M⟧ = v$$

If M evaluates to value v operationally, then M denotes v.

This theorem says our semantics is **correct** — it matches the actual behavior of programs.

In [ ]:
# The adequacy theorem
print("The Adequacy Theorem (EV-882-001):")
print()
print("  Operational semantics: M ⇓ v  (M reduces to value v)")
print("  Denotational semantics: ⟦M⟧ρ = d  (M denotes d)")
print()
print("  Adequacy states these agree:")
print("    M ⇓ v  ⟺  ⟦M⟧ρ = v")
print()
print("  Consequences:")
print("    1. M diverges  ⟺  ⟦M⟧ρ = ⊥")
print("    2. Semantics is compositional")
print("    3. Equivalent denotations ⟹ equivalent behavior")

In [ ]:
# Check adequacy in dataset
adequacy_holds = denotations_df[denotations_df['adequacy_holds'] == True]

print("Expressions where adequacy is verified:")
print()
for _, row in adequacy_holds.head(8).iterrows():
    term_status = "terminates" if row['terminates'] else "diverges"
    print(f"  {row['expression'][:30]:<30} → {row['denotation'][:15]:<15} ({term_status})")

## 7. Compositionality

A key property of denotational semantics is **compositionality**:

> The meaning of a compound expression is determined by the meanings of its parts.

$$⟦M\ N⟧ρ = ⟦M⟧ρ (⟦N⟧ρ)$$

We don't need to look inside M or N — just their denotations.

In [ ]:
# Compositionality example
print("Compositionality in Action:")
print()
print("  Suppose ⟦M⟧ρ = ⟦M'⟧ρ = f  (M and M' have same meaning)")
print()
print("  Then for any N:")
print("    ⟦M N⟧ρ = f(⟦N⟧ρ) = ⟦M' N⟧ρ")
print()
print("  We can substitute M' for M without changing meaning!")
print()
print("  This enables:")
print("    - Modular reasoning")
print("    - Program transformations")
print("    - Compiler optimizations")

## 8. Full Abstraction (The Ongoing Problem)

Adequacy says: same denotation → same behavior.

**Full abstraction** is the converse: same behavior → same denotation.

This is harder! For PCF, full abstraction requires "parallel or" — a function that is sequential in syntax but must be present in semantics.

In [ ]:
# The full abstraction problem
print("The Full Abstraction Problem:")
print()
print("  Adequacy:         ⟦M⟧ = ⟦M'⟧  ⟹  M ≈ M'  ✓")
print("  Full Abstraction: M ≈ M'      ⟹  ⟦M⟧ = ⟦M'⟧  ?")
print()
print("  For PCF with continuous function semantics:")
print("    - Adequacy holds")
print("    - Full abstraction FAILS")
print()
print("  The domain contains 'parallel or' which PCF cannot express.")
print("  por(⊥, true) = true  (returns true without waiting for first arg)")
print()
print("  This remains an active research area.")

## Summary

In this tutorial, we learned:

1. **Denotational semantics** gives mathematical meaning to programs
2. **The semantic function** ⟦M⟧ρ is defined inductively
3. **Variables** look up environments, **abstractions** create functions, **applications** apply
4. **Non-termination** denotes ⊥ — a mathematical meaning for divergence
5. **Recursion** is defined via least fixed points
6. **Adequacy** connects denotational and operational semantics
7. **Compositionality** enables modular reasoning
8. **Full abstraction** remains an open problem

In the final tutorial, we'll synthesize everything into Mott's complete discipline.

---

## Exercises

1. Compute ⟦(λx.λy.x) a b⟧ρ step by step.

2. Why must the semantic function map into domains (not just sets)?

3. Give an example of two expressions with the same denotation but different reduction sequences.

4. What would break if denotational semantics were not compositional?

---

*Next: Tutorial 7 - The Continuous Domain Discipline*